In [90]:
%matplotlib inline

import numpy as np
import pandas as pd
import gzip
import glob
import matplotlib.pyplot as plt
from collections import defaultdict
import itertools
import seaborn as sns
import swifter
from scipy.special import comb

BOTS = ['RandomBot', 'SimpleStatsBot', 'ObserveBot', 'CFRBot_13000000', 'CFRBot_6000000']
NUM_PLAYERS = 5

In [13]:
def load_dataframes():
    result = []
    for filename in glob.glob('tournaments/*.msg.gz'):
        with gzip.open(filename, 'r') as f:
            result.append(pd.read_msgpack(f))
    df = pd.concat(result)
    df.win_type = df.win_type.astype('category')
    df.winner = df.winner.astype('category')
    for i in range(5):
        df['bot_{}'.format(i)] = df['bot_{}'.format(i)].astype('category')
        df['bot_{}_role'.format(i)] = df['bot_{}_role'.format(i)].astype('category')
    df.reset_index(drop=True, inplace=True)
    return df

ALL_GAMES = load_dataframes()

In [52]:
def group_by_bot(games):
    bots = BOTS
    new_dataframe = {}
    for bot in bots:
        print bot
        for i in range(5):
            selector = games['bot_{}'.format(i)] == bot
            if i == 0:
                bot_count = selector.astype(int)
                payoff = games['bot_{}_payoff'.format(i)][selector]
            else:
                bot_count += selector.astype(int)
                payoff = payoff.add(games['bot_{}_payoff'.format(i)][selector], fill_value=0.0)
        new_dataframe['{}_count'.format(bot)] = bot_count
        new_dataframe['{}_payoff'.format(bot)] = payoff.divide(bot_count)

    return pd.DataFrame(new_dataframe)

GROUPED_GAMES = group_by_bot(ALL_GAMES)

CFRBot_13000000
CFRBot_6000000
ObserveBot
RandomBot
SimpleStatsBot


In [55]:
ALL_GAMES[:10]

,bot_0,bot_0_payoff,bot_0_role,bot_1,bot_1_payoff,bot_1_role,bot_2,bot_2_payoff,bot_2_role,bot_3,bot_3_payoff,bot_3_role,bot_4,bot_4_payoff,bot_4_role,win_type,winner
0,ObserveBot,-1.0,merlin,ObserveBot,1.5,assassin,ObserveBot,1.5,minion,SimpleStatsBot,-1.0,servant,CFRBot_6000000,-1.0,servant,Too many bad fails,evil
1,ObserveBot,-1.0,merlin,ObserveBot,1.5,assassin,ObserveBot,1.5,minion,SimpleStatsBot,-1.0,servant,CFRBot_6000000,-1.0,servant,Too many bad fails,evil
2,ObserveBot,-1.0,merlin,ObserveBot,1.5,assassin,ObserveBot,1.5,minion,SimpleStatsBot,-1.0,servant,CFRBot_6000000,-1.0,servant,Too many bad fails,evil
3,ObserveBot,-1.0,merlin,ObserveBot,1.5,assassin,ObserveBot,1.5,minion,SimpleStatsBot,-1.0,servant,CFRBot_6000000,-1.0,servant,Too many bad fails,evil
4,ObserveBot,-1.0,merlin,ObserveBot,1.5,assassin,ObserveBot,1.5,minion,SimpleStatsBot,-1.0,servant,CFRBot_6000000,-1.0,servant,Too many bad fails,evil
5,ObserveBot,-1.0,merlin,ObserveBot,1.5,assassin,ObserveBot,1.5,minion,SimpleStatsBot,-1.0,servant,CFRBot_6000000,-1.0,servant,Too many bad fails,evil
6,ObserveBot,-1.0,merlin,ObserveBot,1.5,assassin,ObserveBot,1.5,minion,SimpleStatsBot,-1.0,servant,CFRBot_6000000,-1.0,servant,Too many bad fails,evil
7,ObserveBot,-1.0,merlin,ObserveBot,1.5,assassin,ObserveBot,1.5,minion,SimpleStatsBot,-1.0,servant,CFRBot_6000000,-1.0,servant,Too many bad fails,evil
8,ObserveBot,-1.0,merlin,ObserveBot,1.5,assassin,ObserveBot,1.5,minion,SimpleStatsBot,-1.0,servant,CFRBot_6000000,-1.0,servant,Too many bad fails,evil
9,ObserveBot,-1.0,merlin,ObserveBot,1.5,assassin,ObserveBot,1.5,minion,SimpleStatsBot,-1.0,servant,CFRBot_6000000,-1.0,servant,Too many bad fails,evil


In [88]:
PAYOFF_TABLE = GROUPED_GAMES.groupby(['{}_count'.format(bot) for bot in BOTS]).mean()
PAYOFF_TABLE = PAYOFF_TABLE[['{}_payoff'.format(bot) for bot in BOTS]].fillna(0.0)

In [68]:
def scipy_multinomial(params):
    if len(params) == 1:
        return 1
    coeff = (comb(np.sum(params), params[-1], exact=True) *
             scipy_multinomial(params[:-1]))
    return coeff

In [71]:
def P(N_i, x):
    x = np.array(x)
    N_i = np.array(N_i)
    return scipy_multinomial(N_i) * np.prod( x ** N_i )

In [93]:
def r(x):
    x = np.array(x)
    numerator = np.zeros(len(x))
    for index, payoff in PAYOFF_TABLE.iterrows():
        numerator += P(index, x) * np.array(payoff)
    denominator = 1.0 - (1.0 - x) ** NUM_PLAYERS
    return numerator / denominator

In [ ]:
LEARNING_RATE = 0.01

def find_nash():
    x = np.array([0.1, 0.1, 0.1, 0.4, 0.3])
    for i in range(100000):
        rx = r(x)
        xtAx = np.sum(x * rx)
        xdot = x * (rx - xtAx)
        x += LEARNING_RATE*xdot
        if i % 100 == 0:
            print x

find_nash()

[0.09973376 0.10000187 0.10018969 0.40004696 0.30002772]
[0.07610228 0.09943428 0.12054992 0.40262874 0.30128477]
[0.05765542 0.09757907 0.14395982 0.40114519 0.29966049]
[0.04337875 0.09471491 0.17093979 0.39569812 0.29526843]
[0.0324041  0.09106591 0.20212437 0.38627488 0.28813074]
[0.02401447 0.08679264 0.23826472 0.37275037 0.2781778 ]
[0.01763171 0.08199032 0.28020911 0.35490848 0.26526038]
[0.01279769 0.07669116 0.32883366 0.33249488 0.24918261]
[0.00915453 0.07087219 0.38487677 0.30532445 0.22977206]
[0.00642643 0.06447529 0.44861807 0.27347089 0.20700932]
[0.00440368 0.057451   0.51937771 0.23754205 0.18122556]
[0.00292812 0.0498332  0.59496531 0.19895588 0.15331749]
[0.00187908 0.04182354 0.6714805  0.15999215 0.12482473]
[0.00116008 0.03382067 0.74396351 0.12337611 0.09767963]
[6.89145059e-04 2.63313821e-02 8.07864764e-01 9.14684559e-02
 7.36462531e-02]
[3.95356370e-04 1.97960352e-02 8.60454504e-01 6.55658116e-02
 5.37882931e-02]
[2.20324329e-04 1.44508644e-02 9.01253405e-01 

/Users/jserrino/.virtualenvs/battlefield/lib/python2.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in divide
  import sys
/Users/jserrino/.virtualenvs/battlefield/lib/python2.7/site-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in subtract
  
/Users/jserrino/.virtualenvs/battlefield/lib/python2.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in multiply
  """


[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan nan nan nan]
[nan nan n